In [1]:
%%capture
# Capture because braindecode module prints a lot :S
from pyFiles.training import *
from pyFiles.models import *
from pyFiles.dataprep_down import *
from pyFiles.dataprep2_down import *
#
import torch
from torch import nn
from torch.nn import functional as F
#
from pyFiles.losses import InfoNceLoss, VICReg, BarlowTwins

In [2]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0" 

In [2]:
#PREP DENUEVO
prep("C:\\Users\\TheSy\\Desktop\\dataverse_files", mode = "per_channel", save = True, save_dir = "data_down")

Data directory created :D
Extracting EDF parameters from C:\Users\TheSy\Desktop\dataverse_files\h01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 231249  =      0.000 ...   924.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 70 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 70.00 Hz: -6.02, -6.02 dB

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
66 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 66 events and 1001 original time points ...
1 bad epochs dropped
Dropped 1 epoch: 64
Extracting EDF parameters from C:\Users\TheSy\Desktop\dataverse_files\h02.edf...
EDF file detected
Setti

,Patient,N_Win,Dir
0,h01,1,data_down\per_channel\sane\h01_w1_ch1.pt
1,h01,1,data_down\per_channel\sane\h01_w1_ch2.pt
2,h01,1,data_down\per_channel\sane\h01_w1_ch3.pt
3,h01,1,data_down\per_channel\sane\h01_w1_ch4.pt
4,h01,1,data_down\per_channel\sane\h01_w1_ch5.pt
...,...,...,...
34043,s14,64,data_down\per_channel\abnormal\s14_w64_ch15.pt
34044,s14,64,data_down\per_channel\abnormal\s14_w64_ch16.pt
34045,s14,64,data_down\per_channel\abnormal\s14_w64_ch17.pt
34046,s14,64,data_down\per_channel\abnormal\s14_w64_ch18.pt


In [3]:
spliter = DFSpliter(save = True, mode= "down_ch")
spliter("down_prep_channels.csv",".")

CSVs creados


(      Patient  N_Win                                             Dir
 0         h04      1        data_down\per_channel\sane\h04_w1_ch1.pt
 1         h04      1        data_down\per_channel\sane\h04_w1_ch2.pt
 2         h04      1        data_down\per_channel\sane\h04_w1_ch3.pt
 3         h04      1        data_down\per_channel\sane\h04_w1_ch4.pt
 4         h04      1        data_down\per_channel\sane\h04_w1_ch5.pt
 ...       ...    ...                                             ...
 26747     s11     64  data_down\per_channel\abnormal\s11_w64_ch15.pt
 26748     s11     64  data_down\per_channel\abnormal\s11_w64_ch16.pt
 26749     s11     64  data_down\per_channel\abnormal\s11_w64_ch17.pt
 26750     s11     64  data_down\per_channel\abnormal\s11_w64_ch18.pt
 26751     s11     64  data_down\per_channel\abnormal\s11_w64_ch19.pt
 
 [26752 rows x 3 columns],
      Patient  N_Win                                             Dir
 0        h10      1        data_down\per_channel\sane\h10_w1_

In [5]:
transforms = Augmentations(5, False, AUGMENTATIONS)

In [6]:
train_dataset = CustomEEGDataset("down_ch_train_feats.csv", root_dir = "", transform = transforms)
val_dataset = CustomEEGDataset("down_ch_val_feats.csv", root_dir = "")

In [7]:
# trainloader = DataLoader(train_dataset, batch_size = 1, shuffle = True, num_workers = 0)
trainloader = DataLoader(train_dataset, batch_size = 350, shuffle = True, num_workers = 0)
valloader = DataLoader(val_dataset, batch_size = 77, shuffle = False, num_workers = 0)

In [8]:
model = SleepStager(n_chans = 19, pretext = False)
enc, fc = torch.load("bestEncoderParamsmulti_inf.pt")
model.encoder.load_state_dict(enc)

criterion = nn.CrossEntropyLoss()

#FREEZE
for param in model.encoder.parameters():
  param.requires_grad = False

optimizer = torch.optim.Adam(model.final.parameters(), lr = 0.001)
downtrain(model, 50, trainloader, valloader, criterion, optimizer, state = None, name = "multi_inf")

Epoch nro 1/5
Iter: 1/5, Loss:0.6911613345146179
Iter: 2/5, Loss:0.928406834602356
Iter: 3/5, Loss:1.5100507736206055
Iter: 4/5, Loss:0.6945882439613342
Iter: 5/5, Loss:0.4950001835823059
Epoch 1/5: Train loss = 0.8638414740562439, Val loss = 1.1302992880344391, Train acc = 0.5042613744735718, Val acc = 0.5
Better params found in epoch = 1, saved params
Epoch nro 2/5
Iter: 1/5, Loss:1.1774204969406128
Iter: 2/5, Loss:0.9615724682807922
Iter: 3/5, Loss:0.8634244203567505
Iter: 4/5, Loss:1.1497396230697632
Iter: 5/5, Loss:0.7583591938018799
Epoch 2/5: Train loss = 0.9821032404899597, Val loss = 0.7144156336784363, Train acc = 0.48721593618392944, Val acc = 0.4921875
Better params found in epoch = 2, saved params
Epoch nro 3/5
Iter: 1/5, Loss:0.6916605234146118
Iter: 2/5, Loss:0.6928603649139404
Iter: 3/5, Loss:0.7274848818778992
Iter: 4/5, Loss:0.7787846922874451
Iter: 5/5, Loss:0.7400315999984741
Epoch 3/5: Train loss = 0.7261644124984741, Val loss = 0.7547358751296998, Train acc = 0.48

([[0.8638414740562439,
   0.9821032404899597,
   0.7261644124984741,
   0.7129442453384399,
   0.6711476087570191],
  [1.1302992880344391,
   0.7144156336784363,
   0.7547358751296998,
   0.7905011892318725,
   0.7708987116813659]],
 [[tensor(0.5043, device='cuda:0'),
   tensor(0.4872, device='cuda:0'),
   tensor(0.4851, device='cuda:0'),
   tensor(0.5355, device='cuda:0'),
   tensor(0.5554, device='cuda:0')],
  [tensor(0.5000, device='cuda:0'),
   tensor(0.4922, device='cuda:0'),
   tensor(0.4661, device='cuda:0'),
   tensor(0.4297, device='cuda:0'),
   tensor(0.3333, device='cuda:0')]])